In [ ]:
!pip install scikit-learn
!pip install numpy==1.16.1

     |████████████████████████████████| 17.3MB 187kB/s 
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.16.1 which is incompatible.
ERROR: pyarrow 3.0.0 has requirement numpy>=1.16.6, but you'll have numpy 1.16.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: astropy 4.2.1 has requirement numpy>=1.17, but you'll have numpy 1.16.1 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


In [ ]:
import numpy as np;
import random;
from sklearn.ensemble import RandomForestClassifier, VotingClassifier;
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score;
import scipy.stats as st;
from scipy.stats import gmean;
import tensorflow as tf

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
np.random.seed(12227)

In [ ]:
def DataPreparation(data_input_file):
  print('Lingjuan Lyu  et al. 2017 {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  X = X[:, 0, :, :]
  
  Y = data['y']
  folds = data['folds']
 
  return X,Y,folds

In [ ]:
def RunLingjuan_Lyu(data_input_file):
  X,Y,folds= DataPreparation(data_input_file)
  avg_acc, avg_recall,avg_f1= Train(X,Y,folds)
  ReportAccuracies(avg_acc, avg_recall,avg_f1)

In [ ]:
def ReportAccuracies(avg_acc, avg_recall,avg_f1):
  ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
  ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
  ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
  print('Mean Accuracy[{:.4f}] G-mean_acc[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), gmean(avg_acc), ic_acc[0], ic_acc[1]))
  print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
  print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))

In [ ]:
def build_model(row,col,num_classes):

  tf.keras.initializers.GlorotNormal(234)
  input_layer = tf.keras.Input(shape=(row,col,1,))
  
  layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(input_layer)
  layer = tf.keras.layers.LSTM(28,return_sequences=True)(layer)
  layer = tf.keras.layers.Dropout(0.2)(layer)
  layer = tf.keras.layers.LSTM(28,return_sequences=True)(layer)
  layer = tf.keras.layers.Dropout(0.2)(layer)
  layerconv1 = tf.keras.layers.Conv1D(filters=30,kernel_size=5,  strides=1) (layer)
  layerconv1 = tf.keras.layers.Activation('relu')(layerconv1)
  layerconv2 = tf.keras.layers.Conv1D(filters=40,kernel_size= 10,strides=1) (layer)
  layerconv2 = tf.keras.layers.Activation('relu')(layerconv2)
  layerconv3 = tf.keras.layers.Conv1D(filters=50,kernel_size= 15,strides=1) (layer)
  layerconv3 = tf.keras.layers.Activation('relu')(layerconv3)
  layerconv4 = tf.keras.layers.Conv1D(filters=60,kernel_size= 20,strides=1) (layer)
  layerconv4 = tf.keras.layers.Activation('relu')(layerconv4)
  max1= tf.reduce_max(layerconv1, 1)
  max2= tf.reduce_max(layerconv2, 1)
  max3= tf.reduce_max(layerconv3, 1)
  max4= tf.reduce_max(layerconv4, 1)
  concat_layer = tf.keras.layers.concatenate([max1,max2,max3,max4],1)
  layer= tf.keras.layers.Dense(num_classes, activation = 'softmax')   (concat_layer) 

    
 
  
    
  model = tf.keras.Model(inputs=input_layer, outputs=layer)
  model.summary()
  return model

In [ ]:
batch_size = 128
epochs = 100

learning_rate = 0.001

In [ ]:
def Train(X,y,folds):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  n_class = y.shape[1]
  _, img_rows, img_cols = X.shape
  X=X.reshape(X.shape[0],img_rows,img_cols,1)
  _,img_rows, img_cols,_ = X.shape
  for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]
    X_train = X[train_idx]
    X_test = X[test_idx]
    
    model=build_model(img_rows, img_cols, n_class)
    optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
   

    model.compile(loss="categorical_crossentropy", optimizer=optimizer,  metrics=['accuracy'])
   
    
    model.fit(X_train, y[train_idx], batch_size=batch_size,epochs=epochs,verbose=2)
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y[test_idx], axis=1)
    acc_fold = accuracy_score(y_true, y_pred)
    avg_acc.append(acc_fold)
    recall_fold = recall_score(y_true, y_pred, average='macro')
    avg_recall.append(recall_fold)
    f1_fold = f1_score(y_true, y_pred, average='macro')
    avg_f1.append(f1_fold)
    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
    print('________________________________________________________________')
    
  return avg_acc, avg_recall,avg_f1

In [ ]:
tf.keras.backend.set_image_data_format('channels_last')